In [1]:
import plotly.express as px
import panel as pn
import pandas as pd
import numpy as np
import os
from pathlib import Path
from dotenv import load_dotenv
import csv
import unicodecsv

In [2]:
# Bring in the Starbucks data
starbucks_df = pd.read_csv(Path('Data_Files/starbucks_locations.csv.txt'))
starbucks_df.head()

,Id,StarbucksId,Name,BrandName,StoreNumber,PhoneNumber,OwnershipType,Street1,Street2,Street3,...,CountrySubdivisionCode,CountryCode,PostalCode,Longitude,Latitude,TimezoneOffset,TimezoneId,TimezoneOlsonId,FirstSeen,LastSeen
0,1f2d204f-e773-4361-9158-0008307dbd5e,10357,Target Virginia T-847,Starbucks,76666-97597,218-741-6603,LS,1001 13th St S,NaN,NaN,...,MN,US,557923254,-92.55,47.51,-360,Central Standard Time,GMT-06:00 America/Chicago,12/8/2013 5:41:59 PM,2/3/2017 12:00:00 AM
1,49455e22-49e9-461f-912c-00092386eaee,6557,PDX A 2,Starbucks,75916-104381,503-284-4008,LS,"7000 NE Airport Way, MB#3",NaN,NaN,...,OR,US,972181031,-122.59,45.59,-480,Pacific Standard Time,GMT-08:00 America/Los_Angeles,12/8/2013 5:41:59 PM,2/3/2017 12:00:00 AM
2,1316ebf8-ee4a-479d-a11d-000964af367b,1010600,235 Bourke Street,Starbucks,25294-240419,NaN,LS,235-251 Bourke St,"Shops 5,6 & 10",NaN,...,VIC,AU,3000,144.97,-37.81,660,AUS Eastern Standard Time,GMT+10:00 Australia/Melbourne,10/8/2014 12:00:00 AM,2/3/2017 12:00:00 AM
3,1b8b9007-4587-4942-bf29-000a0af78161,1014921,Oasis Al-Kharj,Starbucks,27350-247327,NaN,LS,King Abdullah Rd,Al Khuzama,NaN,...,01,SA,NaN,47.27,24.13,180,Arab Standard Time,GMT+03:00 Asia/Riyadh,6/21/2016 12:00:00 AM,2/3/2017 12:00:00 AM
4,072752df-b30b-4f3f-a9b8-000ae32f8666,14667,Target Trumbull T-1956,Starbucks,76306-93245,203-455-0102,LS,120 Hawley Ln,NaN,NaN,...,CT,US,066115347,-73.15,41.23,-300,Eastern Standard Time,GMT-05:00 America/New_York,12/8/2013 5:41:59 PM,2/3/2017 12:00:00 AM


In [3]:
# Drop unnecessary columns

drop_df = starbucks_df.drop(['Id', 'StarbucksId','Name','BrandName','StoreNumber','PhoneNumber','OwnershipType','Street1','Street2','Street3','TimezoneOffset','TimezoneId','TimezoneOlsonId','FirstSeen','LastSeen'], axis = 1)

drop_df

,City,CountrySubdivisionCode,CountryCode,PostalCode,Longitude,Latitude
0,Virginia,MN,US,557923254,-92.55,47.51
1,Portland,OR,US,972181031,-122.59,45.59
2,Melbourne,VIC,AU,3000,144.97,-37.81
3,RIYADH,01,SA,NaN,47.27,24.13
4,Trumbull,CT,US,066115347,-73.15,41.23
...,...,...,...,...,...,...
25595,上海市,31,CN,200001,121.48,31.23
25596,New Taipei City,TPQ,TW,231,121.54,24.98
25597,San Ramon,CA,US,94583,-121.98,37.77
25598,Richmond Hill,ON,CA,L4C 5G5,-79.43,43.85


In [4]:
# Filter all Non-US values/locations

newdf = drop_df.loc[drop_df['CountryCode'] == 'US']
newdf

,City,CountrySubdivisionCode,CountryCode,PostalCode,Longitude,Latitude
0,Virginia,MN,US,557923254,-92.55,47.51
1,Portland,OR,US,972181031,-122.59,45.59
4,Trumbull,CT,US,066115347,-73.15,41.23
5,Oceanside,CA,US,920564672,-117.30,33.18
6,Lone Tree,CO,US,80124,-104.88,39.54
...,...,...,...,...,...,...
25586,Tacoma,WA,US,984097242,-122.47,47.22
25588,Virginia Beach,VA,US,234561410,-76.12,36.79
25593,Wright-Patterson AFB,OH,US,45433541,-84.03,39.81
25597,San Ramon,CA,US,94583,-121.98,37.77


In [5]:
# Sort Dataframe by State
newdf = newdf.sort_values(by = 'CountrySubdivisionCode')

In [6]:
# Sort Dataframe by State and City

newdf = newdf.sort_values(by = ['CountrySubdivisionCode', 'City'])

# Add Column to the Dataframe to capture how many stores are in each city
newdf['Count'] = ''
newdf.head(50)

,City,CountrySubdivisionCode,CountryCode,PostalCode,Longitude,Latitude,Count
7047,Anchorage,AK,US,995152050,-149.86,61.14,
23160,Anchorage,AK,US,99501,-149.89,61.22,
20099,Anchorage,AK,US,995152673,-149.88,61.13,
19113,Anchorage,AK,US,995021401,-149.96,61.14,
8817,Anchorage,AK,US,995084218,-149.87,61.19,
22012,Anchorage,AK,US,99504,-149.74,61.23,
12992,Anchorage,AK,US,995037247,-149.88,61.18,
487,Anchorage,AK,US,995042300,-149.78,61.21,
21665,Anchorage,AK,US,995173340,-149.92,61.19,
1215,Anchorage,AK,US,995073444,-149.84,61.14,


In [33]:
# Get "Average" Latitude for each City

lat_df = newdf.groupby(['CountrySubdivisionCode', 'City'])['Latitude'].mean().reset_index()
lat_df

,CountrySubdivisionCode,City,Latitude
0,AK,Anchorage,61.178333
1,AK,Eagle River,61.333333
2,AK,Fairbanks,64.840000
3,AK,Fort Richardson,61.260000
4,AK,Fort Wainwright,64.830000
...,...,...,...
3756,WY,Jackson Hole,43.480000
3757,WY,Lander,42.840000
3758,WY,Laramie,41.313333
3759,WY,Rock Springs,41.580000


In [34]:
# Get "Average" Longitude for each city
lon_df = newdf.groupby(['CountrySubdivisionCode', 'City'])['Longitude'].mean().reset_index()
lon_df

,CountrySubdivisionCode,City,Longitude
0,AK,Anchorage,-149.856250
1,AK,Eagle River,-149.563333
2,AK,Fairbanks,-147.778000
3,AK,Fort Richardson,-149.680000
4,AK,Fort Wainwright,-147.650000
...,...,...,...
3756,WY,Jackson Hole,-110.750000
3757,WY,Lander,-108.750000
3758,WY,Laramie,-105.570000
3759,WY,Rock Springs,-109.250000


In [7]:
# Count number of Starbucks in each City
count_df = newdf.groupby(['CountrySubdivisionCode', 'City'])['Count'].count().reset_index()
count_df.head(50)

,CountrySubdivisionCode,City,Count
0,AK,Anchorage,24
1,AK,Eagle River,3
2,AK,Fairbanks,5
3,AK,Fort Richardson,1
4,AK,Fort Wainwright,1
5,AK,Homer,1
6,AK,Juneau,2
7,AK,Ketchikan,1
8,AK,Kodiak,1
9,AK,North Pole,1


In [36]:
# Combine Latitude and Longitude Dataframes
combined_geo_df = pd.merge(lon_df, lat_df)
combined_geo_df

,CountrySubdivisionCode,City,Longitude,Latitude
0,AK,Anchorage,-149.856250,61.178333
1,AK,Eagle River,-149.563333,61.333333
2,AK,Fairbanks,-147.778000,64.840000
3,AK,Fort Richardson,-149.680000,61.260000
4,AK,Fort Wainwright,-147.650000,64.830000
...,...,...,...,...
3756,WY,Jackson Hole,-110.750000,43.480000
3757,WY,Lander,-108.750000,42.840000
3758,WY,Laramie,-105.570000,41.313333
3759,WY,Rock Springs,-109.250000,41.580000


In [37]:
# Combine Lat/Lon dataframe with Store Count dataframe
combined_location_df = pd.merge(count_df,combined_geo_df)
combined_location_df

,CountrySubdivisionCode,City,Count,Longitude,Latitude
0,AK,Anchorage,24,-149.856250,61.178333
1,AK,Eagle River,3,-149.563333,61.333333
2,AK,Fairbanks,5,-147.778000,64.840000
3,AK,Fort Richardson,1,-149.680000,61.260000
4,AK,Fort Wainwright,1,-147.650000,64.830000
...,...,...,...,...,...
3756,WY,Jackson Hole,1,-110.750000,43.480000
3757,WY,Lander,1,-108.750000,42.840000
3758,WY,Laramie,3,-105.570000,41.313333
3759,WY,Rock Springs,1,-109.250000,41.580000


In [8]:
# Read in city income information file

pathfile02 = Path('Data_Files/kaggle_income.csv')
income_df = pd.read_csv(pathfile02, encoding='cp1252')
income_df

,id,State_Code,State_Name,State_ab,County,City,Place,Type,Primary,Zip_Code,Area_Code,ALand,AWater,Lat,Lon,Mean,Median,Stdev,sum_w
0,1011000,1,Alabama,AL,Mobile County,Chickasaw,Chickasaw city,City,place,36611,251,10894952,909156,30.771450,-88.079697,38773,30506,33101,1638.260513
1,1011010,1,Alabama,AL,Barbour County,Louisville,Clio city,City,place,36048,334,26070325,23254,31.708516,-85.611039,37725,19528,43789,258.017685
2,1011020,1,Alabama,AL,Shelby County,Columbiana,Columbiana city,City,place,35051,205,44835274,261034,33.191452,-86.615618,54606,31930,57348,926.031000
3,1011030,1,Alabama,AL,Mobile County,Satsuma,Creola city,City,place,36572,251,36878729,2374530,30.874343,-88.009442,63919,52814,47707,378.114619
4,1011040,1,Alabama,AL,Mobile County,Dauphin Island,Dauphin Island,Town,place,36528,251,16204185,413605152,30.250913,-88.171268,77948,67225,54270,282.320328
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32521,720296,72,Puerto Rico,PR,Adjuntas Municipio,Guaynabo,Adjuntas,Track,Track,970,787,589417,1691,18.397925,-66.130633,30649,13729,37977,1321.278082
32522,7202966,72,Puerto Rico,PR,Adjuntas Municipio,Aguada,Adjuntas,Track,Track,602,787,1801613,795887,18.385424,-67.203310,15520,9923,15541,238.813450
32523,7202976,72,Puerto Rico,PR,Adjuntas Municipio,Aguada,Adjuntas,Track,Track,602,787,11031227,0,18.356565,-67.180686,41933,34054,31539,313.551070
32524,7202986,72,Puerto Rico,PR,Adjuntas Municipio,Aguada,Adjuntas,Track,Track,602,787,0,33597561,18.412041,-67.213413,0,0,0,0.000000


In [9]:
# Drop unnecessary columns
income_drop_df = income_df.drop(['id', 'State_Code','County','Place','Type','Primary','Zip_Code','Area_Code','ALand','AWater','Median','Stdev','sum_w'], axis=1)
income_drop_df

,State_Name,State_ab,City,Lat,Lon,Mean
0,Alabama,AL,Chickasaw,30.771450,-88.079697,38773
1,Alabama,AL,Louisville,31.708516,-85.611039,37725
2,Alabama,AL,Columbiana,33.191452,-86.615618,54606
3,Alabama,AL,Satsuma,30.874343,-88.009442,63919
4,Alabama,AL,Dauphin Island,30.250913,-88.171268,77948
...,...,...,...,...,...,...
32521,Puerto Rico,PR,Guaynabo,18.397925,-66.130633,30649
32522,Puerto Rico,PR,Aguada,18.385424,-67.203310,15520
32523,Puerto Rico,PR,Aguada,18.356565,-67.180686,41933
32524,Puerto Rico,PR,Aguada,18.412041,-67.213413,0


In [10]:
#Drop More columns

income_drop_df = income_drop_df.drop(['State_Name','Lat','Lon'], axis=1)
income_drop_df

,State_ab,City,Mean
0,AL,Chickasaw,38773
1,AL,Louisville,37725
2,AL,Columbiana,54606
3,AL,Satsuma,63919
4,AL,Dauphin Island,77948
...,...,...,...
32521,PR,Guaynabo,30649
32522,PR,Aguada,15520
32523,PR,Aguada,41933
32524,PR,Aguada,0


In [11]:
# Sort Dataframe by City & State

income_drop_df.sort_values(by = ['State_ab', 'City'])
income_drop_df

,State_ab,City,Mean
0,AL,Chickasaw,38773
1,AL,Louisville,37725
2,AL,Columbiana,54606
3,AL,Satsuma,63919
4,AL,Dauphin Island,77948
...,...,...,...
32521,PR,Guaynabo,30649
32522,PR,Aguada,15520
32523,PR,Aguada,41933
32524,PR,Aguada,0


In [12]:
# Groupby City/State to get the average income for the city

income_avg_df = income_drop_df.groupby(['State_ab', 'City'])['Mean'].mean().reset_index()

In [23]:
# Round average income to 2 decimals

income_avg_df.round({'Mean': 2})
income_avg_df.head(50)

,CountrySubdivisionCode,City,Mean
0,AK,Alakanuk,40008.000000
1,AK,Allakaket,33847.000000
2,AK,Anchorage,89349.200000
3,AK,Anderson,93374.000000
4,AK,Barrow,94285.000000
5,AK,Bethel,90822.000000
6,AK,Buckland,63175.000000
7,AK,Chevak,51011.000000
8,AK,Chuathbaluk,48313.000000
9,AK,Chugiak,133300.000000


In [14]:
# Rename State column to match the combined Starbucks dataframe for merging

income_avg_df.rename(columns = {'State_ab':'CountrySubdivisionCode'}, inplace = True)

In [24]:
income_avg_df.head(50)

,CountrySubdivisionCode,City,Mean
0,AK,Alakanuk,40008.000000
1,AK,Allakaket,33847.000000
2,AK,Anchorage,89349.200000
3,AK,Anderson,93374.000000
4,AK,Barrow,94285.000000
5,AK,Bethel,90822.000000
6,AK,Buckland,63175.000000
7,AK,Chevak,51011.000000
8,AK,Chuathbaluk,48313.000000
9,AK,Chugiak,133300.000000


In [38]:
# Merge the income and starbucks location dataframes

joined_df = pd.merge(combined_location_df, income_avg_df)

In [39]:
# Verify Columns post-merge

joined_df.columns

Index(['CountrySubdivisionCode', 'City', 'Count', 'Longitude', 'Latitude',
       'Mean'],
      dtype='object')

In [41]:
joined_df.head(50)

,CountrySubdivisionCode,City,Count,Longitude,Latitude,Mean
0,AK,Anchorage,24,-149.856250,61.178333,89349.200000
1,AK,Eagle River,3,-149.563333,61.333333,105052.000000
2,AK,Fairbanks,5,-147.778000,64.840000,79965.250000
3,AK,Homer,1,-151.540000,59.640000,76885.000000
4,AK,Juneau,2,-134.555000,58.360000,86724.000000
5,AK,Ketchikan,1,-131.670000,55.350000,57572.666667
6,AK,Kodiak,1,-152.370000,57.810000,74363.000000
7,AK,North Pole,1,-147.350000,64.760000,98668.000000
8,AK,Palmer,2,-149.125000,61.600000,92824.000000
9,AK,Wasilla,4,-149.417500,61.580000,86586.600000


In [45]:
# Rename Columns to more accurately reflect contents

joined_df.rename(columns = {'CountrySubdivisionCode':'State'}, inplace = True)
joined_df.rename(columns = {'Count':'Store Count'}, inplace = True)
joined_df.rename(columns = {'Mean':'Avg. Income'}, inplace = True)
joined_df

,State,City,Store Count,Longitude,Latitude,Avg. Income
0,AK,Anchorage,24,-149.856250,61.178333,89349.20
1,AK,Eagle River,3,-149.563333,61.333333,105052.00
2,AK,Fairbanks,5,-147.778000,64.840000,79965.25
3,AK,Homer,1,-151.540000,59.640000,76885.00
4,AK,Juneau,2,-134.555000,58.360000,86724.00
...,...,...,...,...,...,...
2912,WY,Gillette,2,-105.490000,44.275000,92205.00
2913,WY,Jackson,2,-110.775000,43.475000,90904.50
2914,WY,Lander,1,-108.750000,42.840000,31047.00
2915,WY,Laramie,3,-105.570000,41.313333,54636.40
